# Storing Memory in a Postgres SQL DB

Langchain message history is now stored as an in-memory dict. Let's store it in a SQL database instead.

In [2]:
# First install postgres then start the service (MacOS)
!brew install postgresql
!brew services start postgresql

# To stop
# !brew services stop postgresql

==> Tapping homebrew/services
Cloning into '/usr/local/Homebrew/Library/Taps/homebrew/homebrew-services'...
remote: Enumerating objects: 2969, done.
remote: Counting objects: 100% (430/430), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 2969 (delta 303), reused 333 (delta 280), pack-reused 2539
Receiving objects: 100% (2969/2969), 821.58 KiB | 7.54 MiB/s, done.
Resolving deltas: 100% (1444/1444), done.
Tapped 1 command (48 files, 1010.4KB).
==> Successfully started `postgresql@14` (label: homebrew.mxcl.postgresql@14)


In [ ]:
# For linux
!sudo apt update
!sudo apt install postgresql postgresql-contrib

# to start
# !sudo service postgresql start

# to stop
# !sudo service postgresql stop

In [ ]:
!pip install psycopg2
# or
!poetry add psycopg2

In [5]:
# create role then database
!psql postgres

# Then run the following commands in the psql shell
# CREATE ROLE reco_admin WITH LOGIN PASSWORD 'averysecurepasswordthatyouwillneverguess';
# ALTER ROLE reco_admin CREATEDB;


# Exit the psql shell then login
# \q
# psql -d postgres -U reco_admin
# \du
# CREATE DATABASE reco WITH OWNER reco_admin ENCODING 'UTF8';
# \c reco
# \q

psql: error: connection to server on socket "/tmp/.s.PGSQL.5432" failed: FATAL:  database "michaelenghoekhor" does not exist


In [ ]:
# To nuke the entire database
# DROP DATABASE reco;

In [7]:
# pip install
!pip install sqlalchemy
!pip install psycopg2-binary
!export LDFLAGS="-L/usr/local/lib"
!export CPPFLAGS="-I/usr/local/include"


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from sqlalchemy import Text, create_engine, Column, Integer, String, DateTime, ForeignKey, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

import typing
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_community.chat_message_histories.sql import DefaultMessageConverter
from langchain_core.messages import BaseMessage, message_to_dict
from sqlalchemy import create_engine
import json


Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True, nullable=False)
    first_name = Column(String(50), nullable=False)
    last_name = Column(String(50), nullable=False)

    # Relationship to enable user.messages backref
    messages = relationship("Message", back_populates="user")

class Message(Base):
    __tablename__ = 'message_store'

    id = Column(Integer, primary_key=True)
    session_id = Column(String(36), index=True, nullable=False)  # UUID format for session_id
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    # typical message is quite long, and we have to account for worst case
    message = Column(Text, nullable=False)
    timestamp = Column(DateTime, server_default=func.now())

    # Relationship to link back to the User
    user = relationship("User", back_populates="messages")

    def __repr__(self):
        return f"Message('{self.session_id}', '{self.user_id}', '{self.message}', '{self.timestamp}')"


# Engine setup (change for your PostgreSQL setup)
USER = 'reco_admin'
PASSWORD = 'averysecurepasswordthatyouwillneverguess'
HOST = 'localhost'
PORT = '5432'
DB = 'reco'
DB_URL = f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}'
engine = create_engine(DB_URL)

/var/folders/qp/hy557z1d5_5ckqwp_fxv8j440000gn/T/ipykernel_64525/1837471835.py:13: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [2]:
Base.metadata.create_all(engine)  # this will create the tables. run this only once

In [3]:
# To create a new session
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
# Test case: Add a new user and a message
new_user = User(username="johndoe", first_name="John", last_name="Doe")
try:
    session.add(new_user)
    session.commit()
except Exception as e:
    print(f"Error: {e}")
    session.rollback()
    new_user = session.query(User).filter_by(username="johndoe").first()


msg_text = '{"type": "ai", "data": {"content": "Hello, I am a bot", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}'
new_message = Message(session_id="123e4567-e89b-12d3-a456-426614174000", user_id=new_user.id, message=msg_text)
session.add(new_message)
session.commit()

# Query the database
user = session.query(User).filter_by(username="johndoe").first()
print(user.messages[0].message)


{"type": "ai", "data": {"content": "Hello, I am a bot", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}


In [8]:
print(user.messages[0])

Message('123e4567-e89b-12d3-a456-426614174000', '1', '{"type": "ai", "data": {"content": "Hello, I am a bot", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}', '2024-06-24 00:46:56.273769')


In [11]:
# Now connecting to LangChain

class CustomMessageConverter(DefaultMessageConverter):
    def __init__(self, user_id: int = None):
        self.user_id = user_id
        self.model_class = Message

    def to_sql_model(self, message: BaseMessage, session_id: str) -> typing.Dict:
        return self.model_class(
            session_id=session_id,
            user_id=self.user_id,
            message=json.dumps(message_to_dict(message))
        )

    def get_sql_model_class(self):
        return Message


def get_session_history(session_id, user_id):
    return SQLChatMessageHistory(
        session_id=session_id, connection=engine,
        custom_message_converter=CustomMessageConverter(user_id=user_id),
    )


In [13]:
# Test case: New session, new user
import uuid

user = User(username="ashketchum", first_name="Ash", last_name="Ketchum")
try:
    session.add(user)
    session.commit()
except Exception as e:
    print(f"Error: {e}")
    session.rollback()
    user = session.query(User).filter_by(username="ashketchum").first()

session_id = uuid.uuid4().hex
chat_history = get_session_history(session_id, user.id)
chat_history.get_messages()

Error: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "users_username_key"
DETAIL:  Key (username)=(ashketchum) already exists.

[SQL: INSERT INTO users (username, first_name, last_name) VALUES (%(username)s, %(first_name)s, %(last_name)s) RETURNING users.id]
[parameters: {'username': 'ashketchum', 'first_name': 'Ash', 'last_name': 'Ketchum'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


[]

In [14]:
chat_history.add_ai_message("Hello Ash, I am a bot")
chat_history.add_user_message("Hello Bot, I am Ash! I wanna be the very best!")
chat_history.add_ai_message("Like no one ever was!")
chat_history.add_user_message("To catch them is my real test!")
chat_history.get_messages()

[AIMessage(content='Hello Ash, I am a bot'),
 HumanMessage(content='Hello Bot, I am Ash! I wanna be the very best!'),
 AIMessage(content='Like no one ever was!'),
 HumanMessage(content='To catch them is my real test!')]

In [15]:
# test retrieval from "cold"
test_return_chat_history = get_session_history(session_id, user.id)
test_return_chat_history.get_messages()

[AIMessage(content='Hello Ash, I am a bot'),
 HumanMessage(content='Hello Bot, I am Ash! I wanna be the very best!'),
 AIMessage(content='Like no one ever was!'),
 HumanMessage(content='To catch them is my real test!')]

## Integrate RunnableWithMessageHistory into our DialogueAgent

In [22]:
from dotenv import load_dotenv
from langchain_core.runnables.history import RunnableWithMessageHistory
from reco_analysis.chatbot.chatbot import DialogueAgent

load_dotenv("../.env")

dialogue_agent = DialogueAgent()

# get Ash
user = session.query(User).filter_by(username="ashketchum").first()

runnable_with_history = RunnableWithMessageHistory(
    dialogue_agent.chain,
    get_session_history=lambda: get_session_history(session_id, user.id),
)

dialogue_agent.chain = runnable_with_history
dialogue_agent.memory = get_session_history(session_id, user.id)

In [23]:
dialogue_agent.get_history()

['Doctor: Hello Ash, I am a bot',
 'Patient: Hello Bot, I am Ash! I wanna be the very best!',
 'Doctor: Like no one ever was!',
 'Patient: To catch them is my real test!']

In [24]:
dialogue_agent.send("Hello Ash, I am definitely not a bot")

In [25]:
dialogue_agent.get_history()

['Doctor: Hello Ash, I am a bot',
 'Patient: Hello Bot, I am Ash! I wanna be the very best!',
 'Doctor: Like no one ever was!',
 'Patient: To catch them is my real test!',
 'Doctor: Hello Ash, I am definitely not a bot']